# Macroblock Image Classification task Using Spark

In [1]:
import os
import numpy as np
import math
import cv2
import re
import pandas as pd

import requests
from PIL import Image
import requests
from io import BytesIO

import findspark, random
findspark.init("/u/cs451/packages/spark")
import os

#Add spark dl package here
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11 --driver-memory 5g pyspark-shell'
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

#Set Config 
sc = SparkContext(appName="YourTest", master= 'local[15]')
spark = SparkSession(sc)

# Requirements for this notebook

In [1]:

!pip3 install nose
!pip3 install pillow
!pip3 install keras
!pip3 install h5py
!pip3 install py4j
!pip3 install tensorflow


  Using cached https://files.pythonhosted.org/packages/15/d8/dd071918c040f50fa1cf80da16423af51ff8ce4a0f2399b7bf8de45ac3d9/nose-1.3.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/10/5c/0e94e689de2476c4c5e644a3bd223a1c1b9e2bdb7c510191750be74fa786/Pillow-6.2.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ad/fd/6bfe87920d7f4fd475acd28500a42482b6b84479832bdc0fe9e589a60ceb/Keras-2.3.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/ab/43e678759326f728de861edbef34b8e2ad1b1490505f20e0d1f0716c3bf4/numpy-1.17.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://fil

# Optional to store Images in HDFS and acessing it from there


In [4]:
#Optional to store Images in HDFS and acessing it from there

# !curl -O http://vision-pc4.eng.uwaterloo.ca/Dataset/H265/Images.tar.gz
# !tar zxvf Images.tar.gz
# hadoop fs -put Images /tmp/Images_MB

Images/
Images/macroblocked/
Images/macroblocked/74.png
Images/macroblocked/3020.png
Images/macroblocked/3747.png
Images/macroblocked/4698.png
Images/macroblocked/1980.png
Images/macroblocked/1514.png
Images/macroblocked/4956.png
Images/macroblocked/3501.png
Images/macroblocked/184.png
Images/macroblocked/2594.png
Images/macroblocked/4853.png
Images/macroblocked/4794.png
Images/macroblocked/1225.png
Images/macroblocked/3464.png
Images/macroblocked/739.png
Images/macroblocked/1149.png
Images/macroblocked/5340.png
Images/macroblocked/3663.png
Images/macroblocked/1443.png
Images/macroblocked/1674.png
Images/macroblocked/5366.png
Images/macroblocked/930.png
Images/macroblocked/4312.png
Images/macroblocked/4812.png
Images/macroblocked/141.png
Images/macroblocked/624.png
Images/macroblocked/3242.png
Images/macroblocked/1661.png
Images/macroblocked/1451.png
Images/macroblocked/4805.png
Images/macroblocked/4130.png
Images/macroblocked/3217.png
Images/macroblocked/4941.png
Images/macroblocked/2

# Data Loading

In [2]:

from sparkdl.image import imageIO
from pyspark.sql.functions import lit

/u8/g3rajasekar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/u8/g3rajasekar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/u8/g3rajasekar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/u8/g3rajasekar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [3]:
#Load from Local
img_dir = "file:///u8/g3rajase/Macroblocking/Images"
pristine_df = imageIO.readImagesWithCustomFn(img_dir + "/pristine",imageIO.PIL_decode).withColumn("label", lit(1))
macro_df = imageIO.readImagesWithCustomFn(img_dir + "/macroblocked",imageIO.PIL_decode).withColumn("label", lit(0))

#You can save this dataframe using spark context and get it using Spark like the cell below

In [4]:
pristine_df = spark.read.format("parquet").load("file:///u8/g3rajase/Macroblocking/pristine.csv")
macro_df = spark.read.format("parquet").load("file:///u8/g3rajase/Macroblocking/macroblocked.csv")

# Train Test Split

In [5]:
pristine_train, pristine_test = pristine_df.randomSplit([0.6, 0.4])
macro_train, macro_test = macro_df.randomSplit([0.6, 0.4])

train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)

train_df.show(1)

In [26]:
#Sampling down test and train data if needed
pristine_train_sampled = pristine_train.sample(False, 0.1, 42)
macro_train_sampled = macro_train.sample(False, 0.1, 42)


pristine_test_sampled = pristine_test.sample(False, 0.1, 42)
macro_test_sampled = macro_test.sample(False, 0.1, 42)


train_df_sampled = pristine_train_sampled.unionAll(macro_train_sampled)
test_df_sampled = pristine_test_sampled.unionAll(macro_test_sampled)

In [27]:
val1 = train_df_sampled.count()
print('Training Data:',val1)

Training Data: 639


In [28]:
val2 = test_df_sampled.count()
print('Testing Data:', val2)

Testing Data: 436


In [29]:
total = val1 + val2
print('Total Data Points:',total)

Total Data Points: 1075


# Spark Training

In [30]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

import time

start = time.time()
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])
p_model = p.fit(train_df_sampled)
end = time.time()-start
# print('Time Taken for Training:',end)

In [31]:
print('Time Taken for Training:',end)

Time Taken for Training: 71.48923540115356


In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
df = p_model.transform(test_df_sampled)
df.show()

predictionAndLabels = df.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Testing set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

+--------------------+-----+--------------------+--------------------+--------------------+----------+
|               image|label|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
|[file:/u8/g3rajas...|    1|[0.0,0.0,0.0,0.0,...|[-3.3628074879755...|[0.03347826100336...|       1.0|
|[file:/u8/g3rajas...|    1|[0.0,0.0,0.0,0.0,...|[-3.1993602955511...|[0.03918980310478...|       1.0|
|[file:/u8/g3rajas...|    1|[0.18046666681766...|[-3.4736921224789...|[0.03007011037714...|       1.0|
|[file:/u8/g3rajas...|    1|[0.22074861824512...|[-3.0692824819744...|[0.04439225589147...|       1.0|
|[file:/u8/g3rajas...|    1|[0.27203428745269...|[-0.6968185407430...|[0.33251797534285...|       1.0|
|[file:/u8/g3rajas...|    1|[0.0,0.6769912242...|[-2.5223984167036...|[0.07430280897966...|       1.0|
|[file:/u8/g3rajas...|    1|[0.0,0.0,0.0,0.0,...|[-3.9799459984385...|[0.

KeyboardInterrupt: 

Results we Got with different config:

Ran using local[*]
1. 651  data trained in  90.82857513427734.  Testing Accuracy is 98.12%
2. 1275 data trained in 130.17509055137634.  Testing Accuracy is 99.1907514450867%
3. 1933 data trained in 189.85703873634338.  Testing Accuracy is 99.24414210128496%
4. 2554 data trained in 249.68797659873962.  Testing Accuracy is 99.251%

Ran using local[5]
1. 651 data trained in  109.549.          Testing Accuracy is 99.07621247113164%
2. 1275 data trained in 132.188818693161. Testing Accuracy is 99.18032786885246%
3. 1933 data trained in 201.167.          Testing Accuracy is 99.00076863950807%
4. 2554 data trained in 256.60.         Testing Accuracy is 99.19507575757576%

Ran using local[15]
1.  651  data trained in 71.4892.                 Testing Accuracy is 98%
2.  1275 data trained in 142.187.                 Testing Accuracy is 98%
3.  1933 data trained in 192.64.                 Testing Accuracy 98is %
4.  2550 data trained in 246.87.           Testing Accuracy is 98.907%

Ran using local[25]
1.  651 data trained in 91.010.           Testing Accuracy is 99.6905311778291%
2.  1275 data trained in 130.1461.        Testing Accuracy is 98.883%
3.  1933 data trained in 191.786.         Testing Accuracy is 98.934%
4.  2550 data trained in 254.618.         Testing Accuracy is 99.13%


Ran without Spark (Python)

1.  651 data trained in 13 mins per epoch.
2.  1275 data trained in 30 mins per epoch.                 
3.  1933 data trained in 55min per epoch.                
4.  2550 data trained in 1hr 20min per epoch.